# Unsafe Code

- People have strong opinions regarding `unsafe` code
- it is the for taking advantage of invariants that the compiler cannot check
- it is not a way to skirt the various rules of Rust, like borrow checking

## The unsafe Keyword

- the `unsafe` keyword is used to
  - mark a function as unsafe (caller must manually ensure that documented invariants hold)
  - declare a block of unsafe code (we must know what we are doing inside that block)
- _Undefined behavior describes_ the consequences of a program that violates invariants of the language at runtime

## Great Power

The additional features are essentially the ability to dereference raw pointers and to call `unsafe` functions

### Juggling Raw Pointers

- declared with `*const T` and `*mut T`
- they don’t have lifetimes and are not subject to the same validity rules of standard `&` references
- you can cast a reference to a pointer
- raw pointers support pointer arithmetic
- many types have utilities to convert to and from raw pointers
- they allow casting a type to another one

### Calling Unsafe Functions

- `unsafe`’s most commonly used feature is that it enables you to call unsafe functions
  - allows calling function implemented within other programming languages like C
  - some unsafe operations can be made safe with additional runtime checks
  - most uses of `unsafe` rely on a variety of custom invariants (e.g.: manual dropping, in-place dropping…)

### Implementing Unsafe Traits

- unsafe traits aren’t unsafe to use, but unsafe to implement
- raw pointers are neither `Send` nor `Sync` (otherwise every type containing a raw pointer would automatically and opaquely `Send`/`Sync`)
- the `GlobalAlloc` trait is an `unsafe` trait because implementors must manually ensure correct alignment of data in memory
- a trait should be unsafe if safe code that assumes that trait is implemented correctly can exhibit memory unsafety if the trait is not implemented correctly (mark a trait as unsafe when you want the implementor to pay additional attention when implementing the trait)
- don’t confuse _safety_ with _correctness_

## Great Responsibility

- the Unsafe Code Guidelines are still a work-in-progress

### What Can Go Wrong?

- `unsafe` code that is not ultimately safe is referred to as having undefined behavior
- the undefined behavior could materialize as either visible crashes or invisible state corruption

### Validity

- references must never dangle, must always be aligned, and must always point to a valid value for their target type
- some primitive type can only assume a finite set of possible values (some compiler optimizations rely on this rule)
- types that point to owned memory (like `Box` and `Vec`) are treated as they held an exclusive reference to such memory
- the `MaybeUninit<T>` type is used to contain potentially invalid data

### Panics

- you must ensure that unsafe code is ready to handle panics
- panics unwind the current thread (recursively drop locals and return from function until the top level function of the stack)
- unsafe operations may produce uninitialized memory that cannot be dropped

### Casting

- two types that are both `#[repr(Rust)]` may have different memory representations, even if they have the same fields
- hence, casting may lead to undefined behavior

### The Drop Check

- when a generic type implements `Drop`, it must be outlived by its arguments
- the `#[may_dangle]` attribute allows to waive that check

## Coping with Fear

### Manage Unsafe Boundaries

- a unsafe code block may have distant effects
- the _privacy boundary_ is everything that interacts with the unsafe bits
- it's better to encapsulate unsafe code as much as possible

### Read and Write Documentation

- ensure to document all the invariants of unsafe code
- repeat global invariant for every function related to them
- ensure to read carefully the documentation of unsafe code

### Check Your Work

- use the interpreted _Miri_ to check for unintended memory operations
- use _sanitizers_ to detect erroneous behavior at runtime
- add many assertions to unsafe code
- Rust safety transitively depends  on the safety of all code (eventually including the unsafe cone). But this also apply to high level languages that adopt the _Ousterhout's dichotomy_
